# Optionally download the circuit without without the edges

In [ ]:
import h5py
import json
import numpy as np
import subprocess
from bluepysnap import Circuit, Simulation
from datetime import datetime, UTC
from entitysdk import Client, ProjectContext, models, types
from entitysdk.staging.circuit import stage_circuit
from entitysdk.staging.simulation_result import stage_simulation, stage_simulation_result
from itertools import product
from libsonata import SimulationConfig
from obi_auth import get_token
from pathlib import Path
from sim_campaign_upload_helpers import generate_scan_config, generate_coordinate_configs

In [ ]:
circuit_id = "0f25396a-1c16-43d7-9943-43e78f84e1ed" # Staging
circuit = "nbS1-O1"
CIRCUIT_ROOT = Path("/Users/james/Documents/obi/Data/Circuits")
circuit_path = CIRCUIT_ROOT / circuit

In [ ]:
import obi_one as obi

from obi_auth import get_token
token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)
circuit = db_client.get_entity(entity_id=circuit_id, entity_type=models.Circuit)

for asset in circuit.assets:
    if asset.label in ["sonata_circuit"]:
        files = db_client.list_directory(
            entity_id=circuit_id, entity_type=models.Circuit, asset_id=asset.id
        )
        for file_path in files.files.keys():
            file_path_str = str(file_path)

            if any(x in file_path_str for x in ["edges", "chemical", "merged-morphologies"]):
                continue

            if "BACKUP" in file_path_str:
                continue

            print(file_path_str)

            db_client.download_file(
                entity_id=circuit_id,
                entity_type=models.Circuit,
                asset_id=asset.id,
                asset_path=file_path_str,
                output_path=circuit_path / file_path_str,
            )